
 # Training visual modality


In [1]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf

import tensorflow.keras as keras

from time import time, gmtime, strftime

from tensorflow.keras.optimizers import Adam

from utils.textUtils.textpreprocessing import FakeDetectionDataTest, FakeDetectionDataTrainVal

from bert.tokenization.bert_tokenization import FullTokenizer

from final_models import create_image_resnet50v2_model

from utils.callbacks.MyCallbacks import MyCallbacks

from utils.datagenUtils.DataSeqThreeModels import DataSequenceThreeModels

from utils.telegramUtils.telegram_bot import telegram_send_message

from utils.textUtils.textpreprocessing import FakeDetectionDataTrainVal, FakeDetectionDataTest

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from utils.models.modelUtils import checkDataframe, loadMeanFromFile, calAccImageModel

In [2]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 32
EPOCHS = 10

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)


# Image Model  Parameters
IMG_WIDTH = 768
IMG_HEIGHT = 768
IMG_DEPTH = 3
IMG_SIZES = (IMG_WIDTH, IMG_HEIGHT)

RETRAIN_TOP_ONLY = False

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

START
Batch Size: 32, Epochs: 10, Optimizer: Adam, Learning Rate; 1e-05, Beta_1: 0.9, Beta_2: 0.999, Epsilon: 1e-08


In [4]:
# Path settings
root = '/home/armin/repos/fkd-model-handling/'

pathToImagesTrain = '/home/armin/repos/FKD-Dataset/002_images/train/' 
pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToImagesVal = '/home/armin/repos/FKD-Dataset/002_images/val/' 
pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToImagesTest = '/home/armin/repos/FKD-Dataset/002_images/test/' 
pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

pathToMeans = '/home/armin/repos/FKD-Dataset/010_configs/means_resized.txt'

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")

In [5]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "ResNet50V2"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [6]:
# Data Preprocessing

df_train = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train['2_way_label'] = df_train['2_way_label'].apply(lambda x: np.array(x))
df_test['2_way_label'] = df_test['2_way_label'].apply(lambda x: np.array(x))
df_val['2_way_label'] = df_val['2_way_label'].apply(lambda x: np.array(x))

# df_train = df_train[:512]
# df_test = df_test[:512]
# df_val = df_val[:512]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)       
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)   

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE


In [7]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

Directory /home/armin/repos/FKD-Dataset/011_checkpoints/ResNet50V2_2020-07-11_15:23/tensorboard does not exist, creating it instead!


In [8]:
meansOfDataset = loadMeanFromFile(pathToMeans, verbose)

In [9]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))

all_text_data = FakeDetectionDataTrainVal(df_train, df_val, tokenizer, [0,1])

max_seq_len = all_text_data.max_seq_len

all_text_test = FakeDetectionDataTest(df_test, tokenizer, [0,1], max_seq_len)

train_x = all_text_data.train_x
val_x = all_text_data.val_x
test_x = all_text_test.test_x

560608it [02:10, 4309.11it/s]
58944it [00:13, 4361.03it/s]


max seq_len 687


58944it [00:13, 4337.14it/s]


max seq_len 512


In [10]:
train_seq = DataSequenceThreeModels(df_train, pathToImagesTrain, train_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)
test_seq = DataSequenceThreeModels(df_test, pathToImagesTest, test_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)
val_seq = DataSequenceThreeModels(df_val, pathToImagesVal,  val_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

In [11]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input , Lambda
from tensorflow.keras import Model
def create_image_resnet50v2_model1(NUM_CLASS, isPreTrained=False, pathToResNet50V2ModelWeights=None):
    resnet50v2 = ResNet50V2(weights='imagenet', include_top=False)
    last_layer = resnet50v2.output
    x = GlobalAveragePooling2D()(last_layer)
    x = Dense(768, activation='relu', name='img_dense_768')(x)
    out = Dense(NUM_CLASS, activation='softmax', name='img_output_layer')(x)
    resnet50v2 = Model(inputs=resnet50v2.input, outputs=out, name="ResNet50V2")
    
    if not isPreTrained:
        return resnet50v2              
    else:
        resnet50v2.load_weights(pathToResNet50V2ModelWeights)
        return resnet50v2, 2

In [12]:
start = time()

mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():
    
    model = create_image_resnet50v2_model1(NUM_CLASS)

    if RETRAIN_TOP_ONLY:
        for layer in model.layers[:-2]:
            layer.trainable = False
        
    if verbose:
        for l in model.layers:
            print(l.name, l.trainable)
        model.summary()
    


    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])
    
    history = model.fit(train_seq,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        epochs=EPOCHS, 
                        validation_data=val_seq,
                        validation_steps=STEP_SIZE_VAL,
                        callbacks=callbacks_list,
                        verbose = TF_VERBOSE
                       )

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task

In [13]:
end = time()
timeProceed = (end - start) / 60
print(f'It took {timeProceed} minutes to train everything' )
telegram_send_message(f'Total time of training: {timeProceed}')

It took 1325.0550244847934 minutes to train everything


In [14]:
plotTimesPerEpoch(callbacks_list)

It took so long to train on one epoch: [278, 274, 271, 252, 250] minutes


In [15]:
calAccImageModel(model, val_seq, 'Val', GLOBAL_BATCH_SIZE)
calAccImageModel(model, test_seq, 'Test', GLOBAL_BATCH_SIZE)

 Val Acc: 0.8089203311617806
 Test Acc: 0.8129241313789359


In [16]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')

done
